## drug2cell

In [ ]:
import scanpy as sc
import drug2cell as d2c
import blitzgsea as blitz

In [ ]:
from time import localtime, strftime
strftime("%Y-%m-%d %H:%M:%S", localtime())

In [ ]:
adata = sc.read(objectDir+"/250521_PPP_integrated_Label_mdc_Labelfine.h5ad") 

In [ ]:
adata

In [ ]:
import pandas as pd
import os

data_file = os.path.expanduser("drug-target_dicts.pkl")

try:
    targets = pd.read_pickle(data_file)
    print("Pickle loaded.")
except FileNotFoundError:
    print(f"file not found: {data_file}")
    exit()

In [ ]:
data = []

for category, drugs in targets.items():
    if isinstance(drugs, dict):
        for key, genes in drugs.items():
            if '|' in key:
                chemb_id, drug_name = key.split('|', 1)
                data.append({
                    'Category': category,
                    'CHEMBL_ID': chemb_id.strip(),
                    'DrugName': drug_name.strip(),
                    'Genes': ','.join(genes)  
                })


df = pd.DataFrame(data)

print(df.head())

In [ ]:
csv_file = os.path.expanduser("drug-target_dicts.csv")
df.to_csv(csv_file, index=False)

In [ ]:
adata = adata[adata.obs["bulk_ident"] == "PPP"].copy()

In [ ]:
d2c.score(adata)

In [ ]:
adata.obs['label_ident'] = adata.obs['label3'].copy()

In [ ]:
print(adata.obs['label_ident'].head())

In [ ]:
adata.obs['label_ident'].unique()

In [ ]:
drug_list = [
    "USTEKINUMAB", "GUSELKUMAB", "TILDRAKIZUMAB", "RISANKIZUMAB",
    "SECUKINUMAB", "IXEKIZUMAB", 'BARICITINIB', 'UPADACITINIB', 'APREMILAST', 'ROFLUMILAST', 'CRISABOROLE',
]

df['DrugName_upper'] = df['DrugName'].str.upper()
filtered_df = df[df['DrugName_upper'].isin(drug_list)]
filtered_df = filtered_df.drop(columns=['DrugName_upper'])
print(filtered_df)

filtered_csv_file = os.path.expanduser("filtered_drug_targets.csv")
filtered_df.to_csv(filtered_csv_file, index=False)

In [ ]:
filtered_csv_file = "filtered_drug_targets.csv"

filtered_df = pd.read_csv(filtered_csv_file)
print(filtered_df.head())

drug_targets = (
    filtered_df
    .groupby("DrugName")["Genes"]
    .apply(lambda rows: list(dict.fromkeys(
        gene.strip() 
        for row in rows
        for gene in row.split(',')
    )))
    .to_dict()
)

for drug, genes in drug_targets.items():
    print(f"{drug}: {genes}")

In [ ]:
min_cells = 3
adata = adata[adata.obs['label_ident'].map(adata.obs['label_ident'].value_counts()) >= min_cells].copy()
d2c.score(adata, targets=drug_targets, nested=False, use_raw=True)

In [ ]:
sc.tl.rank_genes_groups(adata.uns['drug2cell'], method="wilcoxon", groupby="label_ident")
sc.pl.rank_genes_groups_dotplot(adata.uns['drug2cell'], swap_axes=True, dendrogram=False, n_genes=11)

In [ ]:
print(adata.uns['drug2cell'])

In [ ]:
adata.uns['drug2cell'].layers["scaled"] = sc.pp.scale(adata.uns['drug2cell'], copy=True).X

In [ ]:
custom_drug_order = [ "USTEKINUMAB", "GUSELKUMAB", "TILDRAKIZUMAB", "RISANKIZUMAB",
    "SECUKINUMAB", "IXEKIZUMAB", 'BARICITINIB', 'UPADACITINIB', 'APREMILAST', 'ROFLUMILAST', 'CRISABOROLE',]  

In [ ]:
new_order = ['BK-1', 'BK-2','BK-3', 'SBK-1', 'SBK-2', 'SBK-3', 'SBK-4', 'SBK-5', 'Superficial FIB', 'FRC-like FIB', 'Universal FIB', 'Inf universal FIB', 'Schwann-like FIB', 'cDC2-1', 'cDC2-2', 'mDC', 'pDC', 'Macrophage', 'Mast cell', 'CD4n', 'CD4em', 'Th17', 'Treg', 'CD8/NK', 'Endothelial', 'Pericyte', 'Melanocyte']  # replace with your actual labels


adata.uns['drug2cell'].obs['label_ident'] = pd.Categorical(
    adata.uns['drug2cell'].obs['label_ident'],
    categories=new_order,
    ordered=True
)

In [ ]:
sc.pl.dotplot(
    adata.uns['drug2cell'],
    var_names=custom_drug_order,   # list of drug names (rows)
    groupby='label_ident', swap_axes=True, layer="scaled", colorbar_title='z-score', save="250522.dotplot.zscore.drugscore.svg"# column grouping
    )